## Distance Matrix Node - Open Route Service
2 -> 1

Documentatie Open Route Service Distance Matrix: : https://openrouteservice.org/documentation/#/reference/matrix/matrix/matrix-service-(get)  
Documentatie Open Route Service Python: http://openrouteservice-py.readthedocs.io/en/latest/

Vereiste `input_table_1` met origins, en `input_table_2` met destinations.

In [ ]:
# Read from input/output table
import pandas as pd
input_table_1 = pd.read_csv('~/Documents/Open-data/shapefiles/wkt-csv/provincie-centroids.csv')[:3]
input_table_2 = pd.read_csv('~/Documents/Open-data/shapefiles/wkt-csv/gemeente-centroids.csv')[:5]

In [ ]:
# PARAMETERS (OPTIONAL)
# Column for distance matrix index. From input_table_1 (origins)
origins = input_table_1['provincien']
# Column for distance matrix column names. From input_table_2 (destinations)
destinations = input_table_2['GM_NAAM']

# DISTANCE MATRIX
import pandas as pd
import openrouteservice
from openrouteservice.distance_matrix import distance_matrix
from convert import df_to_gdf

#
gdf1 = df_to_gdf(input_table_1)
gdf2 = df_to_gdf(input_table_2)

# Initialize ORS with my personal API key (free token). For production please replace it
client = openrouteservice.Client(
    key='58d904a497c67e00015b45fc45f368a7463145e7be57c7563a900810',
    #base_url='https://foo/bar'
)

# ORS needs 1 list of XY and separates source and destination by a list of indexes
XY = [(Point.x, Point.y) for Point in gdf1.geometry.append(gdf2.geometry)]
idx_origins = range(len(gdf1))
idx_destinations = range(len(gdf1), len(gdf1) + len(gdf2))

# Query ORS
query = distance_matrix(client, XY, sources=idx_origins, destinations=idx_destinations)

# Construct a data frame and convert seconds to minutes
output_table = pd.DataFrame(data=query['durations'], index=origins, columns=destinations)

In [ ]:
# Inspect
output_table